In [1]:
import os

import utils

import models
import models_aimet_high
import models_aimet_medium
import models_aimet_low

import config
import dataset
import dataset_fasdd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import validate

import torch
from torchinfo import summary

import cv2
import matplotlib.pyplot as plt

# Models

## Model No Compression

In [2]:
model_no_comp = models.QUANT_FixedPoint_NoBN_BED_CLASSIFIER(
    weight_bw = config.NO_COMP_WEIGHTS_BIT_WIDTH,
    big_layers_weight_bw = config.NO_COMP_BIG_LAYERS_WEIGHTS_BIT_WIDTH,
    act_bw = config.NO_COMP_ACTIVATIONS_BIT_WIDTH,
    bias_bw = config.NO_COMP_BIAS_BIT_WIDTH,
    num_classes=config.N_CLASSES).to(config.DEVICE)

In [3]:
model_no_comp_folder = './models/'
model_no_comp_name = 'BED_classifier__NOCOMP__smoke__precision=0.9025__recall=0.9021__epoch=35.pt'
model_no_comp_pt = model_no_comp_folder + model_no_comp_name

In [4]:
utils.load_checkpoint(model_path = model_no_comp_pt, 
                      model = model_no_comp, 
                      optimizer= None, 
                      scheduler= None, 
                      device = config.DEVICE)

Loading Model. Trained during 35 epochs


35

## Model Low Compression

In [5]:
model_low_comp = models_aimet_low.QUANT_SOFT_PRUNING_AFTER_SVD_CLASSIFIER(
    weight_bw = config.LOW_COMP_WEIGHTS_BIT_WIDTH,
    big_layers_weight_bw = config.LOW_COMP_BIG_LAYERS_WEIGHTS_BIT_WIDTH,
    act_bw = config.LOW_COMP_ACTIVATIONS_BIT_WIDTH,
    bias_bw = config.LOW_COMP_BIAS_BIT_WIDTH,
    num_classes=config.N_CLASSES).to(config.DEVICE)

In [6]:
model_low_comp_folder = './models/'
model_low_comp_name = 'BED_classifier__LOWCOMP__smoke__precision=0.9024__recall=0.9011__epoch=80.pt'
model_low_comp_pt = model_low_comp_folder + model_low_comp_name

In [7]:
utils.load_checkpoint(model_path = model_low_comp_pt, 
                      model = model_low_comp, 
                      optimizer= None, 
                      scheduler= None, 
                      device = config.DEVICE)

Loading Model. Trained during 80 epochs


80

## Model Medium Compression

In [8]:
model_med_comp = models_aimet_medium.QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER(
    weight_bw = config.MED_COMP_WEIGHTS_BIT_WIDTH,
    big_layers_weight_bw = config.MED_COMP_BIG_LAYERS_WEIGHTS_BIT_WIDTH,
    act_bw = config.MED_COMP_ACTIVATIONS_BIT_WIDTH,
    bias_bw = config.MED_COMP_BIAS_BIT_WIDTH,
    num_classes=config.N_CLASSES).to(config.DEVICE)

In [9]:
model_med_comp_folder = './models/'
model_med_comp_name = 'BED_classifier__MEDCOMP__smoke__precision=0.9028__recall=0.9001__epoch=49.pt'
model_med_comp_pt = model_med_comp_folder + model_med_comp_name

In [10]:
utils.load_checkpoint(model_path = model_med_comp_pt, 
                      model = model_med_comp, 
                      optimizer= None, 
                      scheduler= None, 
                      device = config.DEVICE)

Loading Model. Trained during 49 epochs


49

## Model High Compression

In [11]:
model_high_comp = models_aimet_high.QUANT_PRUNING_AFTER_SVD_CLASSIFIER(
    weight_bw = config.HIGH_COMP_WEIGHTS_BIT_WIDTH,
    big_layers_weight_bw = config.HIGH_COMP_BIG_LAYERS_WEIGHTS_BIT_WIDTH,
    act_bw = config.HIGH_COMP_ACTIVATIONS_BIT_WIDTH,
    bias_bw = config.HIGH_COMP_BIAS_BIT_WIDTH,
    num_classes=config.N_CLASSES).to(config.DEVICE)

In [12]:
model_high_comp_folder = './models/'
model_high_comp_name = 'BED_classifier__HIGHCOMP__smoke__precision=0.9081__recall=0.9006__epoch=90.pt'
model_high_comp_pt = model_high_comp_folder + model_high_comp_name

In [13]:
utils.load_checkpoint(model_path = model_high_comp_pt, 
                      model = model_high_comp, 
                      optimizer= None, 
                      scheduler= None, 
                      device = config.DEVICE)

Loading Model. Trained during 90 epochs


90

# DATASETS

In [14]:
# VALIDATION DATASET
val_transform = A.Compose([
    A.Resize(config.IMG_H, config.IMG_W, p=1),
    ToTensorV2(p=1),
    ]
)

## DFire Val Dataset and Loader

In [15]:
print("\nTEST DFire dataset")
val_dataset = dataset.DFireDataset(
    img_h = config.IMG_H,
    img_w = config.IMG_W,
    img_dir = config.VAL_IMG_DIR,
    label_dir = config.VAL_LABEL_DIR,
    num_classes = config.N_CLASSES,
    ds_len = config.DS_LEN,
    transform=val_transform)

print(f'\nDataset len: {len(val_dataset)}')

# LOADERS
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=config.BATCH_SIZE,
                        num_workers=config.NUM_WORKERS,
                        pin_memory=config.PIN_MEMORY,
                        shuffle=False,
                        drop_last=True)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Dataset len: 4306


## FASDD UAV Dataset

In [16]:
print("\nTEST FASDD UAV dataset")
fasdd_uav_dataset = dataset_fasdd.FASDDDataset(
    img_h = config.IMG_H,
    img_w = config.IMG_W,
    imgs_dir = config.FASDD_UAV_IMGS_DIR,
    labels_file = config.FASDD_UAV_LABELS_FILE,
    num_classes = config.N_CLASSES,
    ds_len = config.DS_LEN,
    transform=val_transform)

print(f'\nDataset len: {len(fasdd_uav_dataset)}')

# LOADERS
fasdd_uav_loader = DataLoader(
    dataset=fasdd_uav_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=True)


TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Dataset len: 4181


## FASDD CV Dataset

In [21]:
print("\nTEST FASDD CV dataset")
fasdd_cv_dataset = dataset_fasdd.FASDDDataset(
    img_h = config.IMG_H,
    img_w = config.IMG_W,
    imgs_dir = config.FASDD_CV_IMGS_DIR,
    labels_file = config.FASDD_CV_LABELS_FILE,
    num_classes = config.N_CLASSES,
    ds_len = config.DS_LEN,
    transform=val_transform)

print(f'\nDataset len: {len(fasdd_cv_dataset)}')

# LOADERS
fasdd_cv_loader = DataLoader(
    dataset=fasdd_cv_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=True)


TEST FASDD CV dataset
DFire Removed wrong images: 0
DFire empty images: 6533
DFire only smoke images: 3902
DFire only fire images: 2091
DFire smoke and fire images: 3358

Dataset len: 15884


# Evaluate all Models

## Run Evaluations with DFire Dataset

In [22]:
print('___________________________ NO COMPRESSION MODEL ___________________________')
metrics_model_no_comp = validate.eval_fn(val_loader, model_no_comp, config.DEVICE)
print('___________________________ LOW COMPRESSION MODEL ___________________________')
metrics_model_low_comp = validate.eval_fn(val_loader, model_low_comp, config.DEVICE)
print('___________________________ MEDIUM COMPRESSION MODEL ___________________________')
metrics_model_med_comp = validate.eval_fn(val_loader, model_med_comp, config.DEVICE)
print('___________________________ HIGH COMPRESSION MODEL ___________________________')
metrics_model_high_comp = validate.eval_fn(val_loader, model_high_comp, config.DEVICE)

___________________________ NO COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:04<00:00, 15.49it/s]


SMOKE -> Precision: 0.9025 - Recall: 0.9021 - Accuracy: 0.9060 - F1: 0.9023
FIRE -> Precision: 0.9352 - Recall: 0.9099 - Accuracy: 0.9604 - F1: 0.9224
Mean F1 Score: 0.9123
___________________________ LOW COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:04<00:00, 16.38it/s]


SMOKE -> Precision: 0.9024 - Recall: 0.9011 - Accuracy: 0.9056 - F1: 0.9018
FIRE -> Precision: 0.9216 - Recall: 0.9324 - Accuracy: 0.9620 - F1: 0.9270
Mean F1 Score: 0.9144
___________________________ MEDIUM COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 16.80it/s]


SMOKE -> Precision: 0.9028 - Recall: 0.9001 - Accuracy: 0.9053 - F1: 0.9015
FIRE -> Precision: 0.9346 - Recall: 0.9144 - Accuracy: 0.9613 - F1: 0.9244
Mean F1 Score: 0.9129
___________________________ HIGH COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:04<00:00, 16.42it/s]

SMOKE -> Precision: 0.9081 - Recall: 0.9006 - Accuracy: 0.9083 - F1: 0.9044
FIRE -> Precision: 0.9416 - Recall: 0.9144 - Accuracy: 0.9632 - F1: 0.9278
Mean F1 Score: 0.9161


## Run Evaluations with FASDD UAV Dataset

In [23]:
print('___________________________ NO COMPRESSION MODEL ___________________________')
metrics_model_no_comp = validate.eval_fn(fasdd_uav_loader, model_no_comp, config.DEVICE)
print('___________________________ LOW COMPRESSION MODEL ___________________________')
metrics_model_low_comp = validate.eval_fn(fasdd_uav_loader, model_low_comp, config.DEVICE)
print('___________________________ MEDIUM COMPRESSION MODEL ___________________________')
metrics_model_med_comp = validate.eval_fn(fasdd_uav_loader, model_med_comp, config.DEVICE)
print('___________________________ HIGH COMPRESSION MODEL ___________________________')
metrics_model_high_comp = validate.eval_fn(fasdd_uav_loader, model_high_comp, config.DEVICE)

___________________________ NO COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 65/65 [00:15<00:00,  4.10it/s]


SMOKE -> Precision: 0.6288 - Recall: 0.8567 - Accuracy: 0.6680 - F1: 0.7253
FIRE -> Precision: 0.9777 - Recall: 0.8535 - Accuracy: 0.9466 - F1: 0.9114
Mean F1 Score: 0.8183
___________________________ LOW COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 65/65 [00:15<00:00,  4.10it/s]


SMOKE -> Precision: 0.5974 - Recall: 0.8773 - Accuracy: 0.6349 - F1: 0.7108
FIRE -> Precision: 0.9696 - Recall: 0.8595 - Accuracy: 0.9462 - F1: 0.9113
Mean F1 Score: 0.8110
___________________________ MEDIUM COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 65/65 [00:15<00:00,  4.10it/s]


SMOKE -> Precision: 0.5958 - Recall: 0.8637 - Accuracy: 0.6305 - F1: 0.7052
FIRE -> Precision: 0.9721 - Recall: 0.8595 - Accuracy: 0.9469 - F1: 0.9123
Mean F1 Score: 0.8087
___________________________ HIGH COMPRESSION MODEL ___________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 65/65 [00:15<00:00,  4.10it/s]

SMOKE -> Precision: 0.6242 - Recall: 0.8694 - Accuracy: 0.6654 - F1: 0.7266
FIRE -> Precision: 0.9794 - Recall: 0.8520 - Accuracy: 0.9466 - F1: 0.9113
Mean F1 Score: 0.8190


## Run Evaluations with FASDD CV Dataset

In [24]:
print('___________________________ NO COMPRESSION MODEL ___________________________')
metrics_model_no_comp = validate.eval_fn(fasdd_cv_loader, model_no_comp, config.DEVICE)
print('___________________________ LOW COMPRESSION MODEL ___________________________')
metrics_model_low_comp = validate.eval_fn(fasdd_cv_loader, model_low_comp, config.DEVICE)
print('___________________________ MEDIUM COMPRESSION MODEL ___________________________')
metrics_model_med_comp = validate.eval_fn(fasdd_cv_loader, model_med_comp, config.DEVICE)
print('___________________________ HIGH COMPRESSION MODEL ___________________________')
metrics_model_high_comp = validate.eval_fn(fasdd_cv_loader, model_high_comp, config.DEVICE)

___________________________ NO COMPRESSION MODEL ___________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████| 248/248 [00:15<00:00, 16.02it/s]


SMOKE -> Precision: 0.5027 - Recall: 0.5928 - Accuracy: 0.5460 - F1: 0.5440
FIRE -> Precision: 0.7078 - Recall: 0.7708 - Accuracy: 0.8124 - F1: 0.7380
Mean F1 Score: 0.6410
___________________________ LOW COMPRESSION MODEL ___________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████| 248/248 [00:15<00:00, 16.34it/s]


SMOKE -> Precision: 0.5263 - Recall: 0.6394 - Accuracy: 0.5723 - F1: 0.5774
FIRE -> Precision: 0.6811 - Recall: 0.8377 - Accuracy: 0.8099 - F1: 0.7513
Mean F1 Score: 0.6643
___________________________ MEDIUM COMPRESSION MODEL ___________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████| 248/248 [00:15<00:00, 16.13it/s]


SMOKE -> Precision: 0.5706 - Recall: 0.6619 - Accuracy: 0.6179 - F1: 0.6129
FIRE -> Precision: 0.7379 - Recall: 0.7921 - Accuracy: 0.8323 - F1: 0.7640
Mean F1 Score: 0.6885
___________________________ HIGH COMPRESSION MODEL ___________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████| 248/248 [00:15<00:00, 16.32it/s]

SMOKE -> Precision: 0.5692 - Recall: 0.6617 - Accuracy: 0.6166 - F1: 0.6120
FIRE -> Precision: 0.7264 - Recall: 0.7962 - Accuracy: 0.8273 - F1: 0.7597
Mean F1 Score: 0.6858
